In [ ]:
from datascience import *
import numpy as np

import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
%matplotlib inline

## The Toast Myth

We saw the Mythbusters crew do an experiment with 48 pieces of toast, where 29 landed butter side up and 19 butter side down.  Let's see if we can figure out how likely this outcome would be, if toast was equally likely to land on either side.  In particular, we'll play a "what-if" game: what if toast was equally likely to land on both sides?  Let's simulate what would happen, under that assumption.

In [ ]:
sides = make_array('Butter Side Up', 'Butter Side Down')

In [ ]:
possible_outcomes = Table().with_column('Outcome', sides)

In [ ]:
possible_outcomes

In [ ]:
simulated_experiment = possible_outcomes.sample(48)

In [ ]:
simulated_experiment

In [ ]:
simulated_experiment.group('Outcome')

In [ ]:
def count_up(sample):
    counts = sample.group('Outcome').where('Outcome', 'Butter Side Up')
    number_up = counts.column('count').item(0)
    return number_up

In [ ]:
count_up(simulated_experiment)

## Simulation

Above we saw how to simulate an episode of the TV show (i.e., one experiment), under the "what-if" assumption that toast is equally likely to land on both sides.  Now we're going to repeat the simulation 10000 times, and keep track of the statistic (the number of times the toast landed butter-side-up) we get from each simulated TV episode.

In [ ]:
counts = make_array()
for i in np.arange(10000): # 10000 repetitions
    one_simulated_episode = possible_outcomes.sample(48)
    number_up = count_up(one_simulated_episode)
    counts = np.append(counts, number_up)
results = Table().with_column('Number that landed butter-side-up', counts)

In [ ]:
results

In [ ]:
results.hist(bins=np.arange(12,36,1))
plots.scatter(29, 0, color='red', s=200)

In [ ]:
results.where('Number that landed butter-side-up',
              are.above_or_equal_to(29)).num_rows / 10000

## Alameda County Juries

In [ ]:
# Data from an ACLU 2010 report
# Racial and Ethnic Disparities in Alameda County Jury Pools
# https://www.aclunc.org/sites/default/files/racial_and_ethnic_disparities_in_alameda_county_jury_pools.pdf

alameda = Table().with_columns(
    'Ethnicity', make_array('Asian', 'Black', 'Latino', 'White', 'Other'),
    'Eligible', make_array(0.15, 0.18, 0.12, 0.54, 0.01),
    'Panels', make_array(0.26, 0.08, 0.08, 0.54, 0.04)
)

alameda.set_format([1, 2], PercentFormatter(0))

In [ ]:
alameda.barh(0)

## Total Variation Distance

In [ ]:
diff = alameda.with_column('Difference',
                    alameda.column('Eligible') - alameda.column('Panels'))
diff

In [ ]:
abs_diff = diff.with_column('Abs. Difference',
                        np.abs(diff.column('Difference')))
abs_diff

In [ ]:
sum(abs_diff.column('Abs. Difference')) / 2

In [ ]:
def total_variation_distance(distribution_1, distribution_2):
    """Each distribution is an array of proportions that sums to 1."""
    return np.abs(distribution_1 - distribution_2).sum()/2

In [ ]:
def table_tvd(table, label_1, label_2):
    return total_variation_distance(table.column(label_1), table.column(label_2))

In [ ]:
table_tvd(alameda, 'Eligible', 'Panels')

## Simulating the statistic

In [ ]:
def get_one_simulated_panel():
    return alameda.select('Ethnicity').sample(1453, weights=alameda.column('Eligible'))

In [ ]:
def simulate_once():
    simulated_panel = get_one_simulated_panel()
    counts = simulated_panel.group('Ethnicity')
    sim_proportions = counts.select('Ethnicity').with_column('Random',
                                    counts.column('count') / 1453)
    sim_proportions.set_format(1, PercentFormatter(0))
    return alameda.join('Ethnicity', sim_proportions)

In [ ]:
simulate_once()

In [ ]:
# Compute the empirical distribution of TVDs

tvds = make_array()

for i in np.arange(1000): # 1000 repetitions of the simulation
    sim_results = simulate_once()
    tvds = np.append(tvds, table_tvd(sim_results, 'Eligible', 'Random'))

results = Table().with_column('TVD', tvds)
results

In [ ]:
results.hist(bins=np.arange(0, 0.2, 0.01))
plots.scatter(0.14, 0, color='red', s=200)

## P-value

In [ ]:
results.where(0, are.above_or_equal_to(0.14)).num_rows / results.num_rows